In [ ]:
!pip install -q -U transformers
!pip install -q -U datasets
!pip install -q -U evaluate
!pip install -q -U tokenizers
!pip install -q -U json-stream
!pip install -q -U jsonlines
!pip install retriv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64

#Setup Environment

In [ ]:
import re
import random
import numpy as np
from scipy.special import softmax
import json
import json_stream
import jsonlines
import torch
import transformers
import evaluate
from tqdm import tqdm
from datasets import load_dataset
import gzip, json, itertools, sys

#Import Wikipedia Backup

In [ ]:
SPLIT          = "full"
CONFIG         = "2019-08-01"
DS_NAME        = "facebook/kilt_wikipedia"
OUT_PATH       = "kilt_paragraphs.jsonl"
SHARD_SIZE     = None

In [ ]:
def stream_paragraphs():
    ds = load_dataset(DS_NAME, CONFIG, split=SPLIT, streaming=True)
    for article in ds:
        title = article["wikipedia_title"]
        wid   = article["wikipedia_id"]
        # article["text"]["paragraph"] is a list of strings  :contentReference[oaicite:0]{index=0}
        for p in article["text"]["paragraph"]:
            yield wid, title, p.strip()

In [ ]:
def write_jsonl():
    out_f = open(OUT_PATH, "wt", encoding="utf‑8")
    for idx, (wid, title, para) in tqdm(
                enumerate(stream_paragraphs(), 1),
                unit_scale=True):
            if re.search("\:{4}", para) == None:
                doc = {
                "id": f"kilt‑{idx}",
                "contents": para,
                }
            else:
                doc = {
                "id": f"kilt‑{idx}",
                "contents": re.split("\:{4}", para)[1].lstrip(",-\s")
                }

            out_f.write(json.dumps(doc, ensure_ascii=False) + "\n")
    out_f.close()

In [ ]:
write_jsonl()

112Mit [5:53:55, 5.26kit/s]


In [ ]:
with jsonlines.open('kilt_paragraphs.jsonl') as reader:
    count = 0
    for obj in reader:
        print(obj['contents'])
        count += 1
        if count > 250:
            break

FileNotFoundError: [Errno 2] No such file or directory: 'kilt_paragraphs.jsonl'

# Generate BM25 Index

In [ ]:
import retriv
from retriv import SparseRetriever

retriv.set_base_path("/content")

In [ ]:
sr = SparseRetriever(
  index_name="new-index",
  model="bm25",
  min_df=1,
  tokenizer="whitespace",
  stemmer="english",
  stopwords="english",
  do_lowercasing=True,
  do_ampersand_normalization=True,
  do_special_chars_normalization=True,
  do_acronyms_normalization=True,
  do_punctuation_removal=True,
)

sr = sr.index_file(
  path="/content/kilt_paragraphs.jsonl",  # File kind is automatically inferred
  show_progress=True,         # Default value
  callback=lambda doc: {"id": doc["id"],"text": doc["contents"]})

## Running required 80GB of system memory to create inverted index
## On an 8-Core CPU, process took 4 hours
## Anything less will cause the process to slow down and crash

Building inverted index: 100%|██████████| 8546202/8546202 [13:14<00:00, 10750.88it/s]


#BM25 Search Example

In [ ]:
sr = SparseRetriever.load("new-index")

In [ ]:
sr.search(
  query="large language models",    # What to search for
  return_docs=True,          # Default value, return the text of the documents
  cutoff=10,                # Default value, number of results to return
)

## BM25 search example

[{'id': 'kilt‑3819832',
  'text': 'A large number of modeling languages appear in the literature.',
  'score': 20.281982},
 {'id': 'kilt‑82332884',
  'text': ' AMPL – modelling language for large-scale linear, mixed integer and nonlinear optimization.',
  'score': 17.73475},
 {'id': 'kilt‑68509174',
  'text': ' Unified Modeling Language, a software modeling language',
  'score': 17.426254},
 {'id': 'kilt‑25332030',
  'text': 'By default, OCRopus comes with a model for English texts and a model for text in Fraktur. These models refer to the script and are largely independent of the actual language. New characters or language variants can be trained either new or in addition.',
  'score': 17.40304},
 {'id': 'kilt‑91912685',
  'text': ' AMPL - a popular modeling language for large-scale linear, mixed integer and nonlinear optimization.',
  'score': 17.194872},
 {'id': 'kilt‑91912638',
  'text': 'Algebraic modeling languages (AML) are high-level computer programming languages for describin